In [8]:
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker import Session
from sagemaker.image_uris import retrieve

# Definir el rol y la sesión
role = get_execution_role()
sess = Session()

# Obtener URI de imagen oficial de SageMaker
image_uri = retrieve(
    framework="xgboost",
    region=sess.boto_region_name,
    version="1.7-1"
)

# Ruta en S3 del archivo tar.gz que contiene el modelo
model_data = 's3://prototype-arep/model.tar.gz'

# Crear el modelo en SageMaker
model = Model(
    model_data=model_data,
    role=role,
    entry_point='inference.py',
    image_uri=image_uri,
    sagemaker_session=sess
)

# Desplegar el modelo
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='xgb-endpoint'
)

[05/13/25 20:00:41] INFO     Ignoring unnecessary instance type: None.                            ]8;id=341908;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=354253;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     Creating model with name: sagemaker-xgboost-2025-05-13-20-00-41-976    ]8;id=610868;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=121316;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[05/13/25 20:00:42] INFO     Creating endpoint-config with name xgb-endpoint                        ]8;id=448595;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=51349;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\

                    INFO     Creating endpoint with name xgb-endpoint                               ]8;id=478554;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=875331;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

--------!

In [6]:
import boto3
import json
region = sess.boto_region_name

client = boto3.client('sagemaker-runtime', region_name=region)

# Ejemplo de datos de entrada
input_data = {
    "productos": [
        {
            "producto": "Leche",
            "stock": 20,
            "temp": 19.77,
            "lluvia": 0.0,
            "estacion": 1  # Puedes usar un mapeo si tu modelo espera 0-3 (0=winter, 1=spring, etc.)
        },
        {
            "producto": "Pan",
            "stock": 15,
            "temp": 20.5,
            "lluvia": 1.2,
            "estacion": 1
        }
    ]
}

response = client.invoke_endpoint(
    EndpointName='xgb-endpoint',
    Body=json.dumps(input_data),
    ContentType='application/json'
)

result = json.loads(response['Body'].read().decode())
print(result)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:27                                                                                   │
│                                                                                                  │
│   24 │   ]                                                                                       │
│   25 }                                                                                           │
│   26                                                                                             │
│ ❱ 27 response = client.invoke_endpoint(                                                          │
│   28 │   EndpointName='xgb-endpoint',                                                            │
│   29 │   Body=json.dumps(input_data),                                                            │
│   30 │   ContentType='application/json'                                                          │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:570 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/context.py:123 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:1031 in    │
│ _make_api_call                                                                                   │
│                                                                                                  │
│   1028 │   │   │   │   "Code"                                                                    │
│   1029 │   │   │   )                                                                             │
│   1030 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1031 │   │   │   raise error_class(parsed_response, opera